In [ ]:
import pandas as pd
import time
import numpy as np
from datetime import datetime
from collections import Counter

In [ ]:
reader = pd.read_csv('./train/train_log.csv',header=0,sep='\t',iterator=True)

In [ ]:
loop = True
chunkSize = 100000
chunks = []
while loop:
    try:
        chunk = reader.get_chunk(chunkSize)
        #print(chunk.head())
        chunk[['day','hour']] = chunk.OCC_TIM.apply(lambda x :pd.Series([i for i in x.split(" ")]))
        chunks.append(chunk)
    except StopIteration:
        loop = False
        print("Iteration is stopped.")
df = pd.concat(chunks, ignore_index=True)

In [ ]:
# 统计一个月的每天点击次数
dict_uid = {}
for i in range(len(df)):
    u_list=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    uid = df.USRID[i]
    day = df.day[i]
    index_day = int(day.split('-')[2])
    if dict_uid.__contains__(uid):
        getlist = dict_uid.get(uid)
        getlist[index_day-1] = getlist[index_day-1] + 1
        dict_uid[uid] = getlist
    else:
        u_list[index_day-1] = 1
        dict_uid[uid] = u_list

In [ ]:
monthDF = pd.DataFrame(dict_uid).T
monthDF['USRID'] = monthDF.index
monthDF.reset_index(drop=True,inplace=True)
columnsList = []
for i in range(31):
    columnsList.append('month_'+str(i))
columnsList.append('USRID')
monthDF.columns = columnsList
monthDF.head()

In [ ]:
# 时间间隔处理
log = df.copy()
log['OCC_TIM_c'] = log['OCC_TIM'].apply(lambda x:time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S")))
log['week'] = log['OCC_TIM'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d %H:%M:%S").weekday())
log = log.sort_values(['USRID','OCC_TIM_c'])
log['next_time'] = log.groupby(['USRID'])['OCC_TIM_c'].diff(-1).apply(np.abs)
log['EVT_LBL_1'] = log['EVT_LBL'].apply(lambda x:x.split('-')[0])
log['EVT_LBL_2'] = log['EVT_LBL'].apply(lambda x:x.split('-')[1])
log['EVT_LBL_3'] = log['EVT_LBL'].apply(lambda x:x.split('-')[2])
time_jiange = log.groupby(['USRID'],as_index=False)['next_time'].agg({
    'next_time_mean':np.mean,
    'next_time_std':np.std,
    'next_time_min':np.min,
    'next_time_max':np.max
})

In [ ]:
# 浏览类别  groupby agg 联合使用
def getLbl(a):
    return len(set(a))
def getLbl_max(a):
    return Counter(list(a)).most_common(1)[0][1]
lbl_1_fea = log.groupby(['USRID'],as_index=False)['EVT_LBL_1'].agg({
    'lbl_1_num':getLbl,
    'lbl_1_maxnum':getLbl_max
})
lbl_2_fea = log.groupby(['USRID'],as_index=False)['EVT_LBL_2'].agg({
    'lbl_2_num':getLbl,
    'lbl_2_maxnum':getLbl_max
})
lbl_3_fea = log.groupby(['USRID'],as_index=False)['EVT_LBL_3'].agg({
    'lbl_3_num':getLbl,
    'lbl_3_maxnum':getLbl_max
})

In [ ]:
lbl_split = pd.merge(pd.merge(lbl_1_fea,lbl_2_fea,on='USRID'),lbl_3_fea,on='USRID')

In [ ]:
# 获得星1到周末的点击量
def getWeek1Features(a):
    try:
        return int(a.value_counts()[0])
    except Exception as e:
        return 0
def getWeek2Features(a):
    try:
        return int(a.value_counts()[1])
    except Exception as e:
        return 0
def getWeek3Features(a):
    try:
        return int(a.value_counts()[2])
    except Exception as e:
        return 0
def getWeek4Features(a):
    try:
        return int(a.value_counts()[3])
    except Exception as e:
        return 0
def getWeek5Features(a):
    try:
        return int(a.value_counts()[4])
    except Exception as e:
        return 0
def getWeek6Features(a):
    try:
        return int(a.value_counts()[5])
    except Exception as e:
        return 0
def getWeek7Features(a):
    try:
        return int(a.value_counts()[6])
    except Exception as e:
        return 0
week1 = log.groupby(['USRID'],as_index=False)['week'].agg({
    'week1':getWeek1Features
})
week2 = log.groupby(['USRID'],as_index=False)['week'].agg({
    'week2':getWeek2Features
})
week3 = log.groupby(['USRID'],as_index=False)['week'].agg({
    'week3':getWeek3Features
})
week4 = log.groupby(['USRID'],as_index=False)['week'].agg({
    'week4':getWeek4Features
})
week5 = log.groupby(['USRID'],as_index=False)['week'].agg({
    'week5':getWeek5Features
})
week6 = log.groupby(['USRID'],as_index=False)['week'].agg({
    'week6':getWeek6Features
})
week7 = log.groupby(['USRID'],as_index=False)['week'].agg({
    'week7':getWeek7Features
})

In [ ]:
week1_2 = pd.merge(week1,week2,on='USRID')
week1_2_3 = pd.merge(week1_2,week3,on='USRID')
week1_2_3_4 = pd.merge(week1_2_3,week4,on='USRID')
week1_2_3_4_5 = pd.merge(week1_2_3_4,week5,on='USRID')
week1_2_3_4_5_6 = pd.merge(week1_2_3_4_5,week6,on='USRID')
week = pd.merge(week1_2_3_4_5_6,week7,on='USRID')

In [ ]:
def timeSplit(dic,str):
    t = int(str.split(':')[0])
    if t>=23 or t<1:
        dic['23-01'] = dic['23-01'] + 1
    if t>=1 and t<3:
        dic['01-03'] = dic['01-03'] + 1
    if t>=3 and t<5:
        dic['03-05'] = dic['03-05'] + 1
    if t>=5 and t<7:
        dic['05-07'] = dic['05-07'] + 1
    if t>=7 and t<9:
        dic['07-09'] = dic['07-09'] + 1
    if t>=9 and t<11:
        dic['09-11'] = dic['09-11'] + 1
    if t>=11 and t<13:
        dic['11-13'] = dic['11-13'] + 1
    if t>=13 and t<15:
        dic['13-15'] = dic['13-15'] + 1
    if t>=15 and t<17:
        dic['15-17'] = dic['15-17'] + 1
    if t>=17 and t<19:
        dic['17-19'] = dic['17-19'] + 1
    if t>=19 and t<21:
        dic['19-21'] = dic['19-21'] + 1
    if t>=21 and t<23:
        dic['21-23'] = dic['21-23'] + 1

In [ ]:
def TCH_TypeSplit(dic,str):
    if str == 0:
        dic['app'] = dic['app'] + 1
    if str == 1:
        dic['web'] = dic['web'] + 1
    if str == 2:
        dic['h5'] = dic['h5'] + 1

In [ ]:
Result_list=[]
for name, group in df.groupby('USRID'):
    dict_x={}
    dict_x['USRID'] = name
    dict_x['MC'] = len(group)
    dict_x['DC'] = len(group.day.drop_duplicates())
    dict_x['CC'] = len(group.EVT_LBL.drop_duplicates())
    dict_t={'23-01':0,'01-03':0,'03-05':0,'05-07':0,'07-09':0,'09-11':0,'11-13':0,'13-15':0,'15-17':0,'17-19':0,'19-21':0,'21-23':0}
    group.hour.map(lambda x:timeSplit(dict_t,x))
    dict_type={'app':0,'web':0,'h5':0}
    group.TCH_TYP.map(lambda x:TCH_TypeSplit(dict_type,x))
    dict_all = {**dict_x, **dict_t,**dict_type}
    Result_list.append(dict_all)   
    

In [ ]:
time_split = pd.DataFrame(Result_list)

In [ ]:
result_a = pd.merge(time_split,time_jiange,how='left',on='USRID')
result_b = pd.merge(result_a,lbl_split,how='left',on='USRID')
result = pd.merge(result_b,week,how='left',on='USRID')

In [ ]:
resut_end = pd.merge(result,monthDF,how='left',on='USRID')

In [ ]:
resut_end.to_csv('./train/train_log_re_4.csv',sep='\t',index=False)

In [ ]:
resut_end.corr()